In [85]:
WIDTH, HEIGHT = (256, 256)
BATCH_SIZE = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  "./photo_samples/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(WIDTH, HEIGHT),
  batch_size=BATCH_SIZE)

# test_ds = train_ds.take(1000)
# train_ds = train_ds.skip(1000)

val_ds = tf.keras.utils.image_dataset_from_directory(
  "./photo_samples/",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(WIDTH, HEIGHT),
  batch_size=BATCH_SIZE)

Found 42 files belonging to 2 classes.
Using 34 files for training.
Found 42 files belonging to 2 classes.
Using 8 files for validation.


In [86]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [87]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [88]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3


2022-09-08 14:30:17.754209: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 1.6563 - accuracy: 0.0000e+00

2022-09-08 14:30:18.826682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 1.6563 - accuracy: 0.0000e+00 - val_loss: 1.3365 - val_accuracy: 0.3750
Epoch 2/3
1/1 [==============================] - 1s 763ms/step - loss: 1.0779 - accuracy: 0.5294 - val_loss: 0.9933 - val_accuracy: 0.6250
Epoch 3/3
1/1 [==============================] - 1s 696ms/step - loss: 1.1715 - accuracy: 0.4706 - val_loss: 0.7770 - val_accuracy: 0.6250


In [90]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(1, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(train_ds)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_ds)


2022-09-08 14:31:09.264640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 6s 6s/step - loss: 0.0000e+00
0 input_16
1 conv2d_861
2 batch_normalization_846
3 activation_846
4 conv2d_862
5 batch_normalization_847
6 activation_847
7 conv2d_863
8 batch_normalization_848
9 activation_848
10 max_pooling2d_51
11 conv2d_864
12 batch_normalization_849
13 activation_849
14 conv2d_865
15 batch_normalization_850
16 activation_850
17 max_pooling2d_52
18 conv2d_869
19 batch_normalization_854
20 activation_854
21 conv2d_867
22 conv2d_870
23 batch_normalization_852
24 batch_normalization_855
25 activation_852
26 activation_855
27 average_pooling2d_81
28 conv2d_866
29 conv2d_868
30 conv2d_871
31 conv2d_872
32 batch_normalization_851
33 batch_normalization_853
34 batch_normalization_856
35 batch_normalization_857
36 activation_851
37 activation_853
38 activation_856
39 activation_857
40 mixed0
41 conv2d_876
42 batch_normalization_861
43 activation_861
44 conv2d_874
45 conv2d_877
46 batch_normalization_859
47 batch_normalization_862
48 act

/Users/jonathankumarich/.virtualenvs/mlops-wwnz/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-09-08 14:31:15.083582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 6s 6s/step - loss: 0.0000e+00
